<a href="https://www.kaggle.com/code/galenchen/taiwan-highway-etc-project?scriptVersionId=125817592" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Kaggle Environment
Just run it so nothing stupid happens.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## File download and zipping
* Make sure all the files within the date of interes is downloaded before doing the analysis.
* This block of code will download all the csv files of the date of interest.
* If the interested day is today, the code will download all the available files.
* The files will be zipped into a zip file for easier download.
* The zip file will be updated if newer files become available and you run the code again.

In [ ]:
import urllib.request
import os
from datetime import datetime
import pytz

def get_all_day_csv(date):
    
    # Check that the input date is valid
    try:
        file_date = pytz.timezone('Asia/Taipei').localize(datetime.strptime(date, '%Y%m%d'))
    except ValueError:
        Downloadable = False
        return('Incorrect date format. Please enter a date in the format YYYYMMDD.')

    # Check that the input date is not in the future
    now = datetime.now(pytz.timezone('Asia/Taipei'))
    delta = now - file_date
    if delta.days < 0:
        Downloadable = False
        return('The specified date is in the future. Please enter a past date.')
    elif delta.days == 0:
        Downloadable = True
        print('Today is not over yet, future files will not be downloaded.')
    else:
        Downloadable = True

    # Initialize variables for counting downloaded files and total file size
    num_downloaded_files = 0
    total_file_size = 0
    
    # Define the total number of files to be downloaded
    num_files = 288
    
    # Loop over dates and hours
    if Downloadable == True: 
        !mkdir /kaggle/working/{date}
        print("Established directory: /kaggle/working/"+str(date))
        
        # Initialize variables for the progress bar
        bar_length = 20
        bar_fill = '█'
        bar_empty = '-'
        bar_progress = 0
        
        # Loop over hours and minutes
        for hour in range(24):
            date_hour = f'{date}/{hour:02d}'
            for minute in range(0, 60, 5):
                file_datetime = pytz.timezone('Asia/Taipei').localize(datetime.strptime(date + f'{hour:02d}{minute:02d}', '%Y%m%d%H%M'))
                if file_datetime > now:
                    break

                time_str = f'{hour:02d}{minute:02d}00'
                filename = f'TDCS_M03A_{date}_{time_str}.csv'
                url = f'https://tisvcloud.freeway.gov.tw/history/TDCS/M03A/{date_hour}/{filename}'
                # Download the file
                if os.path.isfile("/kaggle/working/"+str(date)+"/"+str(filename)):
                    print(f'File {filename} already exists')
                else:
                    urllib.request.urlretrieve(url, filename)
                    if urllib.request.urlopen(url).getcode() == 404:
                        print(f'File {filename} does not exist')
                        os.remove(filename)
                        continue
                    if os.path.getsize(filename) == 0:
                        print(f'File {filename} has no file size')
                        os.remove(filename)
                        continue #make sure it doesn't fuck up

                    # Report the file size and update the counters
                    file_size = os.path.getsize(filename)
                    total_file_size += file_size
                    num_downloaded_files += 1
                    !mv /kaggle/working/{filename} /kaggle/working/{date}/

                    # Update the progress bar
                    bar_progress = int(num_downloaded_files / num_files * bar_length)
                    bar = f'{bar_fill * bar_progress}{bar_empty * (bar_length - bar_progress)}'
                    print(f'Downloading... [{bar}] {num_downloaded_files}/{num_files}', end='\r')

        # Print a summary message of the downloaded files and total file size
        print()
        print(f'Downloaded {num_downloaded_files} files, {total_file_size/1024/1024:.2f} MB in total')

    if Downloadable == False:
        print("Error, file not downloadable.")

    # Zip the downloaded files for easier download.    
    !zip -rq {date}.zip /kaggle/working/{date} > /dev/null
    print("Zipping complete.")
    print()
    print()



## Download CSV in a particular day.

In [ ]:
get_all_day_csv(str(20230318))

## Download several days at once
Remember to enter strings in the list not integers.

In [ ]:
list_of_days = ["10", "11"]

for d in list_of_days:
    day = str(202303)+d #Change year and month as needed.
    get_all_day_csv(day)

### Delete somehing you fucked up.

In [ ]:
date = str(1120318)

!rm /kaggle/working/{date}.zip
!rm /kaggle/working/{date}.csv
!rm -r /kaggle/working/{date}

## Dataframe merging
It will combine all the CSV files into a large Dataframe.

In [ ]:
import os
import pandas as pd
import csv

folder = input("Enter the date of interest: ")

use_input = input("Do you want to use the datasource in ""Input? (y/n): ").lower() == 'y'

if use_input:
    folder_name = f'/kaggle/input/highway-traffic-volume/{folder}/{folder}/'
else:
    # Replace 'folder_name' with the name of the directory containing the CSV files
    folder_name = f'/kaggle/working/{folder}/'

# Get a list of all CSV files in the directory
files = [file for file in os.listdir(folder_name) if file.endswith('.csv')]

def get_file_number(file_name):
    return int(''.join(filter(str.isdigit, file_name)))

# Sort the list of files in ascending numerical order
csv_files = sorted(files, key=get_file_number)

# Initialize an empty list to store the rows
rows = []

# Replace 'file_name.csv' with the name of your CSV file
for file_name in csv_files:
    #print(file_name + " is appended into the list")

    # Open the CSV file and read its contents into the 'rows' list
    with open(f'{folder_name}{file_name}', 'r') as csvfile:
        csvreader = csv.reader(csvfile)
        for row in csvreader:
            rows.append(row)
            
if rows == []:
    print("Error, no files found.")
    
else:
    print("All files in the folder are appended to the list.")
    print()

# Convert the list into a DataFrame
df = pd.DataFrame(rows, columns=['Date_and_time', 'ETC_location', 'Heading', 'Vehicle_class', 'Vehicle_count'])

# Print the DataFrame
print(df)

#Save to pdf
df.to_csv(f"{folder}_merged.csv")

print()
print("The list is saved into csv format.")

## Filter: so you can filter the class of vehicle of interest at a specific etc gate.

In [ ]:

#labels = ['31', '32', '41', '42']
labels = ['31', '32', '41', '42', '5']

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import HourLocator
import pandas as pd

#Assume your original DataFrame is called 'df'
# Extract the rows that meet your conditions
etc = input("ETC of interest, such as 01F3736S")
new_df = df.loc[(df['ETC_location'] == etc) & (df['Vehicle_class'].isin(labels))]

# Create a new DataFrame with the extracted rows
new_df = new_df.copy()

if new_df.empty:
    print("ETC not found.")
    
else:

    # Convert Date_and_time column to datetime type
    new_df['Date_and_time'] = pd.to_datetime(new_df['Date_and_time'])

    # Set up the plot
    fig, ax = plt.subplots(figsize=(16,10))
    ax.set_title(f'Vehicle Count at {etc} on {folder}')
    ax.set_xlabel('Date and Time')
    ax.set_ylabel('Vehicle Count')

    # Convert Vehicle_count column to int type
    new_df['Vehicle_count'] = new_df['Vehicle_count'].astype(int)

    # Plot the data for each vehicle class
    vehicle_classes = ["31", "32", "41", "42", "5"]
    colors = ['r', 'g', 'b', 'y', 'm']
    for i, vehicle_class in enumerate(vehicle_classes):
        class_df = new_df[new_df['Vehicle_class'] == vehicle_class]
        ax.plot(class_df['Date_and_time'], class_df['Vehicle_count'], '-', label=vehicle_class, color=colors[i])

    # Format the x axis
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    ax.xaxis.set_major_locator(HourLocator(interval=1)) # set the major tick interval to 1 hour
    plt.xticks(rotation=45)

    plt.yticks(np.arange(0, max(new_df['Vehicle_count'])+1, 10))
    plt.tick_params(axis='y', labelrotation=0)

    # Set x-axis range to 24 hours
    plt.xlim(new_df['Date_and_time'].iloc[0], new_df['Date_and_time'].iloc[0] + pd.Timedelta(hours=24))

    # Add legend
    ax.legend()

    plt.show()

new_df

new_df.to_csv(f"{folder}_{etc}.csv")

print()
print("The filtered list is saved into csv format.")

## PCU Calculator with customizable weights.

In [ ]:
import pandas as pd

# create a sample dataframe
data = new_df
df_pivoted = pd.DataFrame(data)

# prompt user to choose between default weights or manual input
use_default_weights = input("Do you want to use the default weights? (y/n): ").lower() == 'y'

# get user input for class weights
if use_default_weights:
    class_weights = {'31': 1.0, '32': 1.0, '41': 1.5, '42': 1.5, '5': 3.0}
else:
    class_weights = {}
    for vc in ["31", "32", "41", "42", "5"]:
        weight = float(input(f"Enter weight for class {vc}: "))
        class_weights[vc] = weight

# create a new dataframe to store the pivoted data
result = pd.DataFrame()

# add the 'Time' column with the time in the format of HHMM
result['Time'] = [f'{h:02d}{m:02d}' for h in range(0, 24) for m in range(0, 60, 5)]

# loop through the vehicle class values and pivot the data for each group of 5 rows
for vc in ["31", "32", "41", "42", "5"]:
    # filter the dataframe to include only the rows with the current vehicle class
    temp = df_pivoted[df_pivoted['Vehicle_class'] == vc][['Vehicle_count']]
    # rename the column to include the vehicle class value
    temp = temp.rename(columns={'Vehicle_count': vc})
    # reset the index of the pivoted data
    temp = temp.reset_index(drop=True)
    # append the pivoted data to the result dataframe
    result = pd.concat([result, temp], axis=1)

# calculate the weighted total for each time interval
result['Weighted_total_PCU'] = 12 * (result['31']*class_weights['31'] + result['32']*class_weights['32'] + 
                            result['41']*class_weights['41'] + result['42']*class_weights['42'] +
                            result['5']*class_weights['5'])

# calculate average of three weighted totals and output it in the third column, and then 2-4, 3-5... 
# until the last 3 rows are averaged
result['Avg15_PCU'] = result['Weighted_total_PCU'].rolling(3).mean().shift(0).round(0)

# Avg60, same thing,
result['Avg60_PCU'] = result['Weighted_total_PCU'].rolling(12).mean().shift(0).round(0)

result.to_csv(f"{folder}_{etc}_PCU.csv")
print("The filtered list is saved into csv format.")
print()
result


## PCU plot generator.

In [ ]:
import matplotlib.pyplot as plt

# set the figure size
plt.figure(figsize=(16, 10))

# plot the Weighted_total column
plt.plot(result['Time'], result['Weighted_total_PCU'], label='Weighted Total PCU', color='blue')

# plot the Avg15 column
plt.plot(result['Time'], result['Avg15_PCU'], label='Avg15 PCU', color='green')

# plot the Avg60 column
plt.plot(result['Time'], result['Avg60_PCU'], label='Avg60 PCU', color='red')

# set the x-ticks to show only every hour
plt.xticks(result['Time'][::12])
plt.xticks(rotation=45)

# add title and legend
plt.title(f'Weighted Vehicle Traffic on at {etc} on {folder}')
plt.legend()

plt.yticks(np.arange(0, max(result['Weighted_total_PCU'])+1, 500))
plt.tick_params(axis='y', labelrotation=0)

# display the plot
plt.show()
